# NIFTY 50 Daily Direction Prediction (MVP)

Goal:
Predict whether NIFTY 50 will go up (1) or down (0) tomorrow using historical data and ML.

In [ ]:
!pip install yfinance pandas numpy matplotlib scikit-learn


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
ticker = "^NSEI"

df = yf.download(
    ticker,
    start="2006-01-01",
    auto_adjust=False,
    progress=False
)

# Fix possible MultiIndex columns
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)

df = df.dropna()
df.head()


Price,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2007-09-17,4494.649902,4494.649902,4549.049805,4482.850098,4518.450195,0
2007-09-18,4546.200195,4546.200195,4551.799805,4481.549805,4494.100098,0
2007-09-19,4732.350098,4732.350098,4739.000000,4550.250000,4550.250000,0
2007-09-20,4747.549805,4747.549805,4760.850098,4721.149902,4734.850098,0
2007-09-21,4837.549805,4837.549805,4855.700195,4733.700195,4752.950195,0


In [ ]:
df["target"] = (df["Close"].shift(-1) > df["Close"]).astype(int)
df = df.iloc[:-1]


In [ ]:
df[["Close", "target"]].head(10)


Price,Close,target
Date,,
2007-09-17,4494.649902,1
2007-09-18,4546.200195,1
2007-09-19,4732.350098,1
2007-09-20,4747.549805,1
2007-09-21,4837.549805,1
2007-09-24,4932.200195,1
2007-09-25,4938.850098,1
2007-09-26,4940.500000,1
2007-09-27,5000.549805,1


In [ ]:
df["target"].value_counts(normalize=True)


,proportion
target,
1,0.530195
0,0.469805


In [ ]:
type(df.index)


pandas.core.indexes.datetimes.DatetimeIndex

FEATURE ENGINEERING

In [ ]:
# Daily log return
df["log_return"] = np.log(df["Close"] / df["Close"].shift(1))

# Rolling returns (momentum)
df["ret_5"]  = df["Close"].pct_change(5)
df["ret_10"] = df["Close"].pct_change(10)
df["ret_20"] = df["Close"].pct_change(20)


In [ ]:
# Moving averages
df["ma_5"]   = df["Close"].rolling(5).mean()
df["ma_10"]  = df["Close"].rolling(10).mean()
df["ma_20"]  = df["Close"].rolling(20).mean()
df["ma_50"]  = df["Close"].rolling(50).mean()

# Price relative to trend
df["price_ma_10"] = df["Close"] / df["ma_10"] - 1
df["price_ma_20"] = df["Close"] / df["ma_20"] - 1
df["price_ma_50"] = df["Close"] / df["ma_50"] - 1


In [ ]:
# Rolling volatility
df["vol_5"]  = df["log_return"].rolling(5).std()
df["vol_10"] = df["log_return"].rolling(10).std()
df["vol_20"] = df["log_return"].rolling(20).std()


In [ ]:
# Rolling min/max
rolling_min_20 = df["Close"].rolling(20).min()
rolling_max_20 = df["Close"].rolling(20).max()

# Position in recent range (0 = bottom, 1 = top)
df["range_pos_20"] = (df["Close"] - rolling_min_20) / (rolling_max_20 - rolling_min_20)


In [ ]:
df = df.dropna()


In [ ]:
df.head()

Price,Adj Close,Close,High,Low,Open,Volume,target,log_return,ret_5,ret_10,...,ma_10,ma_20,ma_50,price_ma_10,price_ma_20,price_ma_50,vol_5,vol_10,vol_20,range_pos_20
Date,,,,,,,,,,,,,,,,,,,,,
2007-11-27,5698.149902,5698.149902,5743.549805,5655.600098,5729.250000,0,0,-0.005871,-0.014314,0.000483,...,5756.42500,5777.994995,5440.556992,-0.010123,-0.013819,0.047347,0.023883,0.022489,0.017239,0.427189
2007-11-28,5617.549805,5617.549805,5749.950195,5595.500000,5699.549805,0,1,-0.014246,0.010160,-0.053950,...,5724.38999,5765.434985,5463.014990,-0.018664,-0.025650,0.028287,0.015830,0.017348,0.017438,0.234619
2007-11-29,5634.600098,5634.600098,5725.000000,5612.100098,5617.799805,0,1,0.003031,0.020881,-0.046938,...,5696.63999,5752.132495,5484.782988,-0.010891,-0.020433,0.027315,0.014915,0.017560,0.017392,0.275356
2007-11-30,5762.750000,5762.750000,5782.549805,5632.649902,5633.899902,0,1,0.022489,0.027485,-0.024395,...,5682.22998,5746.947485,5505.390986,0.014170,0.002750,0.046747,0.016406,0.019580,0.018223,0.581532
2007-12-03,5865.000000,5865.000000,5878.799805,5754.600098,5765.450195,0,0,0.017588,0.023257,-0.007219,...,5677.96499,5743.577490,5527.739990,0.032941,0.021141,0.061012,0.015459,0.020590,0.018500,0.825827


In [ ]:
df.tail()

Price,Adj Close,Close,High,Low,Open,Volume,target,log_return,ret_5,ret_10,...,ma_10,ma_20,ma_50,price_ma_10,price_ma_20,price_ma_50,vol_5,vol_10,vol_20,range_pos_20
Date,,,,,,,,,,,,,,,,,,,,,
2026-01-20,25232.500000,25232.500000,25585.000000,25171.349609,25580.300781,409800,0,-0.013893,-0.021626,-0.038773,...,25758.009961,25942.500098,25957.892148,-0.020402,-0.027368,-0.027945,0.005668,0.005311,0.005496,0.000000
2026-01-21,25157.500000,25157.500000,25300.949219,24919.800781,25141.000000,395600,1,-0.002977,-0.022338,-0.039009,...,25655.890039,25891.755078,25949.555156,-0.019426,-0.028359,-0.030523,0.005608,0.005305,0.005044,0.000000
2026-01-22,25289.900391,25289.900391,25435.750000,25168.500000,25344.150391,486400,0,0.005249,-0.014638,-0.032549,...,25570.805078,25847.392578,25941.454180,-0.010985,-0.021569,-0.025116,0.007163,0.006033,0.005279,0.113061
2026-01-23,25048.650391,25048.650391,25347.949219,25025.300781,25344.599609,393900,1,-0.009585,-0.025130,-0.032005,...,25487.985156,25792.720117,25924.911172,-0.017237,-0.028848,-0.033800,0.007242,0.005965,0.005562,0.000000
2026-01-27,25175.400391,25175.400391,25246.650391,24932.550781,25063.349609,618700,1,0.005047,-0.016029,-0.019776,...,25437.195117,25749.375098,25910.836172,-0.010292,-0.022291,-0.028383,0.008581,0.006282,0.005770,0.099031


In [ ]:
df.columns


Index(['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume', 'target',
       'log_return', 'ret_5', 'ret_10', 'ret_20', 'ma_5', 'ma_10', 'ma_20',
       'ma_50', 'price_ma_10', 'price_ma_20', 'price_ma_50', 'vol_5', 'vol_10',
       'vol_20', 'range_pos_20'],
      dtype='object', name='Price')

In [ ]:
df[[
    "Close",
    "ret_5",
    "price_ma_20",
    "vol_10",
    "range_pos_20",
    "target"
]].head()


Price,Close,ret_5,price_ma_20,vol_10,range_pos_20,target
Date,,,,,,
2007-11-27,5698.149902,-0.014314,-0.013819,0.022489,0.427189,0
2007-11-28,5617.549805,0.010160,-0.025650,0.017348,0.234619,1
2007-11-29,5634.600098,0.020881,-0.020433,0.017560,0.275356,1
2007-11-30,5762.750000,0.027485,0.002750,0.019580,0.581532,1
2007-12-03,5865.000000,0.023257,0.021141,0.020590,0.825827,0


In [ ]:
# Drop raw moving averages (keep ratios only)
drop_cols = ["ma_5", "ma_10", "ma_20", "ma_50"]
df = df.drop(columns=drop_cols)


In [ ]:
# Separate features and target
X = df.drop(columns=["target"])
y = df["target"]

print(X.shape, y.shape)


(4455, 17) (4455,)


In [ ]:
# Time-based split
split_date = "2019-01-01"

X_train = X.loc[X.index < split_date]
X_test  = X.loc[X.index >= split_date]

y_train = y.loc[y.index < split_date]
y_test  = y.loc[y.index >= split_date]

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)


Train size: (2711, 17)
Test size: (1744, 17)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)


In [ ]:
df["prev_direction"] = (df["Close"].diff() > 0).astype(int)


Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    random_state=42
)

model_lr.fit(X_train_scaled, y_train)


LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model_lr.predict(X_test_scaled)

acc = accuracy_score(y_test, y_pred)
print("Logistic Regression Accuracy:", round(acc * 100, 2), "%")


Logistic Regression Accuracy: 50.11 %


Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=6,              # prevents overfitting
    min_samples_leaf=50,      # very important for finance
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)


RandomForestClassifier(max_depth=6, min_samples_leaf=50, n_estimators=300,
                       n_jobs=-1, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score

rf_pred = rf_model.predict(X_test)
rf_acc = accuracy_score(y_test, rf_pred)

print("Random Forest Accuracy:", round(rf_acc * 100, 2), "%")


Random Forest Accuracy: 53.61 %


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, rf_pred)


array([[189, 611],
       [198, 746]])

In [ ]:
from sklearn.metrics import accuracy_score

# Predictions on TRAINING data
rf_train_pred = rf_model.predict(X_train)

# Training accuracy
train_acc = accuracy_score(y_train, rf_train_pred)

print("Random forest Training Accuracy:", round(train_acc * 100, 2), "%")
print(f"Training Accuracy : {train_acc*100:.2f}%")
print(f"Test Accuracy     : {rf_acc*100:.2f}%")



Random forest Training Accuracy: 66.14 %
Training Accuracy : 66.14%
Test Accuracy     : 53.61%


In [ ]:
# Confidence thresholds
upper_threshold = 0.6
lower_threshold = 0.4


Two Day Direction

In [ ]:
horizon = 2

# Define feature_columns based on the features used for the previous model (X)
feature_columns = X.columns.tolist()

# 2-day ahead target
df["target_2d"] = (df["Close"].shift(-horizon) > df["Close"]).astype(int)

# features + target
df_rf_2d = df[feature_columns + ["target_2d"]].dropna()

In [ ]:
X_rf = df_rf_2d[feature_columns]
y_rf = df_rf_2d["target_2d"]


In [ ]:
X_train_rf = X_rf.loc[X_rf.index < split_date]
X_test_rf  = X_rf.loc[X_rf.index >= split_date]

y_train_rf = y_rf.loc[y_rf.index < split_date]
y_test_rf  = y_rf.loc[y_rf.index >= split_date]

print("Train size:", X_train_rf.shape)
print("Test size:", X_test_rf.shape)


Train size: (2711, 17)
Test size: (1744, 17)


In [ ]:
y_rf.value_counts(normalize=True)


,proportion
target_2d,
1,0.540741
0,0.459259


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_2d = RandomForestClassifier(
    n_estimators=600,
    max_depth=8,
    min_samples_leaf=150,   # IMPORTANT for time-series
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)

rf_2d.fit(X_train_rf, y_train_rf)


RandomForestClassifier(class_weight='balanced', max_depth=8,
                       min_samples_leaf=150, n_estimators=600, n_jobs=-1,
                       random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score

rf_pred = rf_2d.predict(X_test_rf)
rf_acc = accuracy_score(y_test_rf, rf_pred)

print("Random Forest 2-Day Test Accuracy:", round(rf_acc * 100, 2), "%")


Random Forest 2-Day Test Accuracy: 55.33 %


In [ ]:
import joblib

joblib.dump(rf_2d, "rf_nifty_2day_model.pkl")
joblib.dump(feature_columns, "feature_columns.pkl")

print("RF 2-day model and feature schema saved.")


RF 2-day model and feature schema saved.


In [ ]:
def compute_features(input_df):
    df_copy = input_df.copy()

    # Daily log return
    df_copy["log_return"] = np.log(df_copy["Close"] / df_copy["Close"].shift(1))

    # Rolling returns (momentum)
    df_copy["ret_5"]  = df_copy["Close"].pct_change(5)
    df_copy["ret_10"] = df_copy["Close"].pct_change(10)
    df_copy["ret_20"] = df_copy["Close"].pct_change(20)

    # Moving averages
    df_copy["ma_5"]   = df_copy["Close"].rolling(5).mean()
    df_copy["ma_10"]  = df_copy["Close"].rolling(10).mean()
    df_copy["ma_20"]  = df_copy["Close"].rolling(20).mean()
    df_copy["ma_50"]  = df_copy["Close"].rolling(50).mean()

    # Price relative to trend
    df_copy["price_ma_10"] = df_copy["Close"] / df_copy["ma_10"] - 1
    df_copy["price_ma_20"] = df_copy["Close"] / df_copy["ma_20"] - 1
    df_copy["price_ma_50"] = df_copy["Close"] / df_copy["ma_50"] - 1

    # Rolling volatility
    df_copy["vol_5"]  = df_copy["log_return"].rolling(5).std()
    df_copy["vol_10"] = df_copy["log_return"].rolling(10).std()
    df_copy["vol_20"] = df_copy["log_return"].rolling(20).std()

    # Rolling min/max
    rolling_min_20 = df_copy["Close"].rolling(20).min()
    rolling_max_20 = df_copy["Close"].rolling(20).max()

    # Position in recent range (0 = bottom, 1 = top)
    df_copy["range_pos_20"] = (df_copy["Close"] - rolling_min_20) / (rolling_max_20 - rolling_min_20)

    # Drop raw moving averages (keep ratios only)
    drop_cols = [col for col in ["ma_5", "ma_10", "ma_20", "ma_50"] if col in df_copy.columns]
    if drop_cols: # Only drop if columns exist
        df_copy = df_copy.drop(columns=drop_cols, errors='ignore')

    df_copy = df_copy.dropna()

    return df_copy

def predict_next_2days_yes_no(df, model, feature_columns):

    #returns YES or NO (no abstain).


    # compute features using full history
    df_feat = compute_features(df)

    # latest available trading day (after market close)
    latest_row = df_feat.iloc[-1][feature_columns].to_frame().T

    # probability market goes UP after 2 days
    prob_up = model.predict_proba(latest_row)[0, 1]


    if prob_up >= 0.5:
        return "YES, market grows UP after 2 days", prob_up
    else:
        return "NO. market falls DOWN after 2 days", prob_up

In [ ]:
decision, probability = predict_next_2days_yes_no(
    df,
    rf_2d,
    feature_columns
)

print("Decision -->", decision)
print("Probability -->", round(probability, 3))



Decision --> YES, market grows UP after 2 days
Probability --> 0.54


In [ ]:
# Download RELIANCE stock data
df_rel = yf.download(
    "RELIANCE.NS",
    start="2006-01-01",
    auto_adjust=False,
    progress=False
)

# Fix MultiIndex if present
if isinstance(df_rel.columns, pd.MultiIndex):
    df_rel.columns = df_rel.columns.get_level_values(0)

df_rel = df_rel.dropna()
df_rel.head()


Price,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2006-01-02,55.198135,64.111961,64.408295,63.562130,63.765640,51281633
2006-01-03,56.559875,65.693604,65.972084,63.979858,64.229774,68687753
2006-01-04,57.165440,66.396950,66.836098,65.693604,65.765007,60512596
2006-01-05,56.664387,65.814995,66.836098,65.693604,66.621880,74344078
2006-01-06,56.602913,65.743584,66.511200,65.200897,65.814995,89054027


In [ ]:
df_rel_feat = compute_features(df_rel)
df_rel_feat = df_rel_feat.dropna()


In [ ]:
horizon = 2

df_rel_feat["target_2d"] = (
    df_rel_feat["Close"].shift(-horizon) > df_rel_feat["Close"]
).astype(int)

df_rel_feat = df_rel_feat.iloc[:-horizon]


In [ ]:
X_rel = df_rel_feat[feature_columns]
y_rel = df_rel_feat["target_2d"]


In [ ]:
X_train_rel = X_rel.loc[X_rel.index < split_date]
X_test_rel  = X_rel.loc[X_rel.index >= split_date]

y_train_rel = y_rel.loc[y_rel.index < split_date]
y_test_rel  = y_rel.loc[y_rel.index >= split_date]

print("Train size:", X_train_rel.shape)
print("Test size:", X_test_rel.shape)


Train size: (3154, 17)
Test size: (1747, 17)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_rel_2d = RandomForestClassifier(
    n_estimators=400,
    max_depth=7,
    min_samples_leaf=80,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)

rf_rel_2d.fit(X_train_rel, y_train_rel)


RandomForestClassifier(class_weight='balanced', max_depth=7,
                       min_samples_leaf=80, n_estimators=400, n_jobs=-1,
                       random_state=42)

In [ ]:
pred_rel = rf_rel_2d.predict(X_test_rel)
acc_rel = accuracy_score(y_test_rel, pred_rel)

print("RELIANCE 2-Day Test Accuracy:", round(acc_rel * 100, 2), "%")


RELIANCE 2-Day Test Accuracy: 49.17 %


In [ ]:
# Create a copy of test data for backtesting
bt_df = df.loc[X_test_rf.index].copy()

# Add model predictions (YES = 1, NO = 0)
bt_df["signal"] = rf_2d.predict(X_test_rf)

# Trend filter: price above 200-day MA
bt_df["ma_150"] = df.loc[bt_df.index, "Close"].rolling(150).mean()

# Apply trend filter
bt_df["final_signal"] = (
    (bt_df["signal"] == 1) &
    (bt_df["Close"] > bt_df["ma_150"])
).astype(int)

In [ ]:
# Create a copy of test data for backtesting
bt_df = df.loc[X_test_rf.index].copy()

# Add model predictions (YES = 1, NO = 0)
bt_df["signal"] = rf_2d.predict(X_test_rf)

bt_df.head()


Price,Adj Close,Close,High,Low,Open,Volume,target,log_return,ret_5,ret_10,...,price_ma_10,price_ma_20,price_ma_50,vol_5,vol_10,vol_20,range_pos_20,prev_direction,target_2d,signal
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,10792.500000,10792.500000,10895.349609,10735.049805,10868.849609,309700,0,-0.006470,0.012097,-0.008803,...,-0.003185,0.001728,0.016802,0.005654,0.008072,0.009810,0.634959,0,0,1
2019-01-03,10672.250000,10672.250000,10814.049805,10661.250000,10796.799805,286200,1,-0.011205,-0.005368,-0.021675,...,-0.012134,-0.008525,0.004774,0.007714,0.008617,0.010102,0.383836,0,1,1
2019-01-04,10727.349609,10727.349609,10741.049805,10628.650391,10699.700195,296600,1,0.005150,-0.004866,-0.021879,...,-0.004823,-0.003149,0.009046,0.007811,0.008597,0.010045,0.498903,1,1,1
2019-01-07,10771.799805,10771.799805,10835.950195,10750.150391,10804.849609,269400,1,0.004135,-0.008112,-0.016427,...,0.000971,0.000188,0.012037,0.007032,0.008830,0.009273,0.591730,1,1,1
2019-01-08,10802.150391,10802.150391,10818.450195,10733.250000,10786.250000,277700,1,0.002814,-0.005560,0.004477,...,0.003342,0.002502,0.013789,0.007292,0.006694,0.009101,0.655112,1,1,1


In [ ]:
# Trend filter: price above 200-day MA
bt_df["ma_200"] = df.loc[bt_df.index, "Close"].rolling(200).mean()

# Apply trend filter
bt_df["final_signal"] = (
    (bt_df["signal"] == 1) &
    (bt_df["Close"] > bt_df["ma_200"])
).astype(int)

bt_df["market_return"] = bt_df["Close"].pct_change(horizon)
bt_df["strategy_return"] = bt_df["market_return"] * bt_df["signal"]

bt_df["strategy_return_filtered"] = (
    bt_df["market_return"] * bt_df["final_signal"]
)

bt_df = bt_df.dropna()

In [ ]:
# Cumulative returns
bt_df["cum_market"] = (1 + bt_df["market_return"]).cumprod()
bt_df["cum_strategy"] = (1 + bt_df["strategy_return"]).cumprod()

bt_df[["cum_market", "cum_strategy"]].tail()


Price,cum_market,cum_strategy
Date,,
2026-01-20,4.624389,16.840422
2026-01-21,4.547032,16.558711
2026-01-22,4.557375,16.596380
2026-01-23,4.537657,16.524572
2026-01-27,4.517113,16.449757


In [ ]:
# Sharpe ratio (annualized, approx)
sharpe = (
    bt_df["strategy_return"].mean()
    / bt_df["strategy_return"].std()
) * np.sqrt(252)

# Maximum drawdown
rolling_max = bt_df["cum_strategy"].cummax()
drawdown = bt_df["cum_strategy"] / rolling_max - 1
max_dd = drawdown.min()

print("Sharpe Ratio:", round(sharpe, 2))
print("Max Drawdown:", round(max_dd * 100, 2), "%")


Sharpe Ratio: 2.09
Max Drawdown: -57.05 %


In [ ]:
# Total number of years in backtest
years = (bt_df.index[-1] - bt_df.index[0]).days / 365.25

# Annualized return
annual_return = bt_df["cum_strategy"].iloc[-1] ** (1 / years) - 1

print("Annualized Return:", round(annual_return * 100, 2), "%")


Annualized Return: 56.61 %


In [ ]:
calmar = annual_return / abs(max_dd)

print("Calmar Ratio:", round(calmar, 2))


Calmar Ratio: 0.99


In [ ]:
import os

base_dirs = [
    "data",
    "notebooks",
    "models",
    "src"
]

for d in base_dirs:
    os.makedirs(d, exist_ok=True)

print("Project folders created!")


Project folders created!
